# Sentiment Analysis with Amazon Reviews

+ Work with the open-sourced Amazon Review data-set to experiment with Sentiment Analysis on customer reviews.  
+ Use Databricks SQL AI functions and Databricks foundation model serving enpoints.  
+ Reference: [Databricks Blog: step-step-guide-ai-powered-customer-sentiment-analysis](https://www.databricks.com/blog/step-step-guide-ai-powered-customer-sentiment-analysis)  

This example shows:
1. Predicting the `rating` (1..5) that a customer will give a product, based on the review text
2. Classification of the product being reviewed, based on the review text
3. Explanation of why the classification catagegory was chosen.
  
Approach is to use a foundation LLM via the Databricks model serving service and query it using the SQL AI Query function.  
A few-shot chat prompt template is used to structured the request and output.

### Parameterize the Catalog and Schema Location


In [0]:
dbutils.widgets.text("catalog", "users")
dbutils.widgets.text("database", "")
dbutils.widgets.text("table", "reviews")

In [0]:
print("Table location params set to", dbutils.widgets.getAll())
catalog = dbutils.widgets.get("catalog")
database = dbutils.widgets.get("database")
table = dbutils.widgets.get("table")

In [0]:
# Check which pre-loaded data-sets are available
#display(dbutils.fs.ls("/databricks-datasets/"))

## Load Sample Dataset to `{catalog}.{database}.{table}`

In [0]:
# Amazon Reviews sample data-set
display(dbutils.fs.ls("/databricks-datasets/amazon/data20K/"))

In [0]:
df = spark.read.format("parquet").load("/databricks-datasets/amazon/data20K/")
display(df.head(5))

In [0]:
df.write.format("delta").mode("overwrite").saveAsTable(f"{catalog}.{database}.{table}")

In [0]:
%sql
SELECT * FROM $catalog.$database.$table LIMIT 10;

## Create a Features Table with RowIDs

The `review_features` table has a unique ID added for each row.  Use this later for comparing our predicted reviews with the actual Amazon review. 

In [0]:
%sql
CREATE TABLE ${catalog}.${database}.review_features AS
SELECT 
  ROW_NUMBER() OVER (ORDER BY (SELECT NULL)) AS id,
  *
FROM 
  ${catalog}.${database}.${table};

In [0]:
%sql
SELECT * from ${catalog}.${database}.review_features
LIMIT 10;

# Prompted LLM to Predict Review Scores and Review Categories


### Use `ai_query` with a custom prompt for Few Shot Learning

[ai_query](https://learn.microsoft.com/en-gb/azure/databricks/sql/language-manual/functions/ai_query) 

+ This function is not available on Azure Databricks SQL Classic.
+ Databricks Runtime 15.4 or above is recommended. 
+ Your workspace must be in a supported Model Serving region.
+ Needs an existing model serving endpoint with your model loaded. If you are using a Databricks hosted foundation model, an endpoint is created for you. Otherwise, see Create custom model serving endpoints or Create foundation model serving endpoints.
+ Querying Foundation Model APIs is enabled by default. To query endpoints that serve custom models or external models, enable AI_Query for Custom Models and External Models in the Databricks Previews UI.


In [0]:
%sql
WITH extracted_opinions_cte AS (
  SELECT
    id,
    review,
    rating,
    ai_query(
      --"databricks-claude-sonnet-4",
      "databricks-meta-llama-3-3-70b-instruct",
      request => concat(
        "You are an opinion mining service. Given a piece of text, output a struct with the following fields: classification, sentiment.

          For the classification field, categorize the text as one of the following 10 categories:
          CLOTHING, BOOKS, MUSIC, MOVIES_TV, ELECTRONICS, TOYS_GAMES, TOOLS_HOME, SPORTS_OUTDOORS, OTHER, UNKNOWN.

          - CLOTHING: Apparel, shoes, fashion accessories.
          - BOOKS: Any printed or digital book.
          - MUSIC: CDs, Vinyls, or music-related products.
          - MOVIES_TV: DVDs, Blu-rays, or TV/movies-related content.
          - ELECTRONICS: Any gadget, device, or electronic item.
          - TOYS_GAMES: Toys, board games, or children's games.
          - TOOLS_HOME: Tools, home improvement, kitchenware, furniture.
          - SPORTS_OUTDOORS: Gear for sports, fitness, or outdoor activities.
          - OTHER: Clearly identified products that don’t fit the above.
          - UNKNOWN: If there is not enough context to classify.

          The sentiment field is a score from 1 to 5:
          - 5 = Very Positive
          - 4 = Positive
          - 3 = Neutral
          - 2 = Negative
          - 1 = Very Negative

          If you are not confident about the classification, make sure to say it is UNKNOWN; do not guess.  If there is doubt, go for UNKNOWN
          Stick to the existing categories, do not add new ones.

          Output format must be:
          JSON: {'classification': '<CATEGORY>', 'sentiment': <SENTIMENT_SCORE>, 'classification_reason': '<REASON>'}

          Return only the JSON result. Do not include the original docoment in the output. Only include the final answer in the JSON format specified.


          ### EXAMPLES:

          DOCUMENT  
          This mouse is amazing, I had owned a Razer Naga, and a R.A.T. 7 mouse. And I am not afraid to say that this mouse takes the cake, for $20 it looks like its worth $80. The only issue i had was that the seller sent me a wireless version, but I don't mind because it works just as well. Also where you see silver on the mouse in reality it is glossy black, and the mouse looks better IRL then in the pictures. I will add on to this review in a month or so to check in.  
          RESULT  
          {'classification': 'ELECTRONICS', 'sentiment': 5, 'classification_reason': 'The text is about electronics.'}

          DOCUMENT  
          Fabric is nice and soft but zipper broke the first time we used it. Very disappointing. The fit was fine, so we still use it to get our monies worth  
          RESULT  
          {'classification': 'CLOTHING', 'sentiment': 2, 'classification_reason': 'The text is about clothing.'}


          DOCUMENT  
          Worked as expected. I'm not sure what else you expect me to say. I expected no less. Dunno what else to say.  
          RESULT  
          {'classification': 'UNKNOWN', 'sentiment': 4, 'classification_reason': 'The text is not clear enough to classify.'}

          DOCUMENT  
          I love this product and the other items that go with it. Bath and Body dropped this scent so I was thrilled to find that I could still purchase it.  
          RESULT  
          {'classification':'OTHER', 'sentiment': 5, 'classification_reason': 'The text is about perfume which is not one of the categories I am using.'}

          ### INCORRECT EXAMPLE:

          RESULT
          {'classification': 'FURNITURE', 'sentiment': 4, 'classification_reason': 'The text is about furniture, which falls under the category TOOLS_HOME.'} 

          Corrected response:
          {'classification': 'TOOLS_HOME', 'sentiment': 4, 'classification_reason': 'The text is about furniture, which falls under the category TOOLS_HOME.'}

          This is wrong because only the Corrected response should be provided.

          ",
          review
      )
    ) as extracted_opinions
  FROM
    ${catalog}.${database}.review_features
)
SELECT
  review,
  get_json_object(extracted_opinions, '$.classification') AS classification,
  rating,
  get_json_object(extracted_opinions, '$.sentiment') AS predicted_rating,
  get_json_object(extracted_opinions, '$.classification_reason') AS classification_reason
FROM
  extracted_opinions_cte
LIMIT 10;

**The next part takes about 10 minutes to execute**. (process about 20,000 rows of data)

In [0]:
%sql
CREATE TABLE ${catalog}.${database}.review_predictions 
AS
WITH extracted_opinions_cte AS (
  SELECT
    id,
    review,
    ai_query(
      --"databricks-claude-sonnet-4",
      "databricks-meta-llama-3-3-70b-instruct",
      request => concat(
        "You are an opinion mining service. Given a piece of text, output a struct with the following fields: classification, sentiment.

          For the classification field, categorize the text as one of the following 10 categories:
          CLOTHING, BOOKS, MUSIC, MOVIES_TV, ELECTRONICS, TOYS_GAMES, TOOLS_HOME, SPORTS_OUTDOORS, OTHER, UNKNOWN.

          - CLOTHING: Apparel, shoes, fashion accessories.
          - BOOKS: Any printed or digital book.
          - MUSIC: CDs, Vinyls, or music-related products.
          - MOVIES_TV: DVDs, Blu-rays, or TV/movies-related content.
          - ELECTRONICS: Any gadget, device, or electronic item.
          - TOYS_GAMES: Toys, board games, or children's games.
          - TOOLS_HOME: Tools, home improvement, kitchenware, furniture.
          - SPORTS_OUTDOORS: Gear for sports, fitness, or outdoor activities.
          - OTHER: Clearly identified products that don’t fit the above.
          - UNKNOWN: If there is not enough context to classify.

          The sentiment field is a score from 1 to 5:
          - 5 = Very Positive
          - 4 = Positive
          - 3 = Neutral
          - 2 = Negative
          - 1 = Very Negative

          If you are not confident about the classification, make sure to say it is UNKNOWN; do not guess.  If there is doubt, go for UNKNOWN
          Stick to the existing categories, do not add new ones.

          Output format must be:
          JSON: {'classification': '<CATEGORY>', 'sentiment': <SENTIMENT_SCORE>, 'classification_reason': '<REASON>'}

          Return only the JSON result. Do not include the original docoment in the output. Only include the final answer in the JSON format specified.


          ### EXAMPLES:

          DOCUMENT  
          This mouse is amazing, I had owned a Razer Naga, and a R.A.T. 7 mouse. And I am not afraid to say that this mouse takes the cake, for $20 it looks like its worth $80. The only issue i had was that the seller sent me a wireless version, but I don't mind because it works just as well. Also where you see silver on the mouse in reality it is glossy black, and the mouse looks better IRL then in the pictures. I will add on to this review in a month or so to check in.  
          RESULT  
          {'classification': 'ELECTRONICS', 'sentiment': 5, 'classification_reason': 'The text is about electronics.'}

          DOCUMENT  
          Fabric is nice and soft but zipper broke the first time we used it. Very disappointing. The fit was fine, so we still use it to get our monies worth  
          RESULT  
          {'classification': 'CLOTHING', 'sentiment': 2, 'classification_reason': 'The text is about clothing.'}


          DOCUMENT  
          Worked as expected. I'm not sure what else you expect me to say. I expected no less. Dunno what else to say.  
          RESULT  
          {'classification': 'UNKNOWN', 'sentiment': 4, 'classification_reason': 'The text is not clear enough to classify.'}

          DOCUMENT  
          I love this product and the other items that go with it. Bath and Body dropped this scent so I was thrilled to find that I could still purchase it.  
          RESULT  
          {'classification':'OTHER', 'sentiment': 5, 'classification_reason': 'The text is about perfume which is not one of the categories I am using.'}

          ### INCORRECT EXAMPLE:

          RESULT
          {'classification': 'FURNITURE', 'sentiment': 4, 'classification_reason': 'The text is about furniture, which falls under the category TOOLS_HOME.'} 

          Corrected response:
          {'classification': 'TOOLS_HOME', 'sentiment': 4, 'classification_reason': 'The text is about furniture, which falls under the category TOOLS_HOME.'}

          This is wrong because only the Corrected response should be provided.

          ",
          review
      )
    ) as extracted_opinions
  FROM
    ${catalog}.${database}.review_features
)
SELECT
  id,
  review,
  get_json_object(extracted_opinions, '$.sentiment') AS rating,
  get_json_object(extracted_opinions, '$.classification') AS classification,
  get_json_object(extracted_opinions, '$.classification_reason') AS classification_reason
FROM
  extracted_opinions_cte


In [0]:
%sql
SELECT 
  p.id, p.review, 
  p.rating as predicted_rating, 
  f.rating as actual_rating, 
  p.classification as predicted_classification,
  p.classification_reason as predicted_classification_reason
FROM 
  ${catalog}.${database}.review_predictions p 
JOIN 
  ${catalog}.${database}.review_features f 
  ON f.id = p.id
LIMIT 50;

In [0]:
%sql
SELECT 
  p.rating - f.rating as diff, 
  COUNT(*) as count,
  ROUND(100*COUNT(*)/19959) as percent -- there are 19959 rows in the dataset
FROM 
  ${catalog}.${database}.review_predictions p 
JOIN 
  ${catalog}.${database}.review_features f 
ON 
  p.id = f.id
GROUP BY 
  p.rating - f.rating
ORDER BY 
  diff;

Databricks visualization. Run in Databricks to view.

In [0]:
%python
# Load the review_predictions table
review_predictions_df = spark.table("users.ed_bullen.review_predictions")

# Sample 30 rows randomly
sampled_df = review_predictions_df.sample(withReplacement=False, fraction=0.1).limit(10)

# Display the sampled rows
display(sampled_df)

## Conclusion

Results for `databricks-meta-llama-3-3-70b-instruct`:

+ 71% of the predicted ratings have the same rating prediction as was given by the Amazon customer
+ 97% are within +/- 1 rating of the Amazon customer


Review Classification predictions is lacking ground truths for quantitative evaluation, but appears to be giving sensible results.